<a href="https://colab.research.google.com/github/mfilipak/AFRAC_IA/blob/main/002_Embeddings1_Portal_da_Transpar%C3%AAncia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Experimento 002
##Objetivo: Experimentos com word embeddings usando como corpo o campo de descrição do dataset
###Descrição: Experimentos iniciais para criação e visualização de word embeddings